# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy.stats import percentileofscore as score

## Importing List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks (1).csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2557516310449,
 'week52high': 162.72,
 'week52low': 108.16,
 'week52highSplitAdjustOnly': 160.81,
 'week52lowSplitAdjustOnly': 111.03,
 'week52change': 0.2870738253582676,
 'sharesOutstanding': 16781039796,
 'float': 0,
 'avg10Volume': 75376671,
 'avg30Volume': 79079437,
 'day200MovingAvg': 143.51,
 'day50MovingAvg': 152.3,
 'employees': 151576,
 'ttmEPS': 5.1,
 'ttmDividendRate': 0.887418373239105,
 'dividendYield': 0.005952566460536803,
 'nextDividendDate': '',
 'exDividendDate': '2021-07-30',
 'nextEarningsDate': '2021-10-18',
 'peRatio': 29.4837190925988,
 'beta': 1.367816505097503,
 'maxChangePercent': 56.46940891729784,
 'year5ChangePercent': 4.458373686651761,
 'year2ChangePercent': 1.627126063493555,
 'year1ChangePercent': 0.3052213174226001,
 'ytdChangePercent': 0.13251900460580332,
 'month6ChangePercent': 0.10807370320119793,
 'month3ChangePercent': 0.04686497393844006,
 'month1ChangePercent': 0.018714276357230844,
 'day30ChangePerce

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.3052213174226001

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

# for symbol_string in symbol_strings:
#         print(symbol_string)

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe =  final_dataframe.append(
        pd.Series(
        [
            symbol, 
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'

        ],
            index= my_columns
        ),
        ignore_index = True
)


In [8]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,159.60,0.495012,N/A
1,AAL,19.92,0.567332,N/A
2,AAP,227.51,0.488341,N/A
3,AAPL,156.04,0.305383,N/A
4,ABBV,112.10,0.343683,N/A
...,...,...,...,...
500,YUM,131.56,0.317276,N/A
501,ZBH,155.29,0.0571684,N/A
502,ZBRA,542.94,0.822578,N/A
503,ZION,63.34,1.20847,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [9]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace= True )
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,148,DVN,41.20,3.748,N/A
1,317,MRO,17.03,3.19244,N/A
2,175,FANG,116.41,3.14791,N/A
3,357,OXY,34.00,2.44846,N/A
4,275,LB,80.00,2.34224,N/A
5,299,MCHP,77.50,1.79632,N/A
6,410,SIVB,730.18,1.64587,N/A
7,251,IT,332.37,1.62554,N/A
8,195,FTNT,339.46,1.60867,N/A
9,23,ALB,244.97,1.58357,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [10]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the size of your portfolio:")
    
    try:
        float(portfolio_size)
    except:
        print("Enter a valid Number..!")
        portfolio_input()

portfolio_input()
print(portfolio_size)

Enter the size of your portfolio:100000
100000


In [11]:
position_size = float(portfolio_size) /len(final_dataframe.index)
for i in range(0,len(final_dataframe)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Price'])
final_dataframe


C:\Users\Aniket Bhanderi\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,148,DVN,41.20,3.748,48
1,317,MRO,17.03,3.19244,117
2,175,FANG,116.41,3.14791,17
3,357,OXY,34.00,2.44846,58
4,275,LB,80.00,2.34224,25
5,299,MCHP,77.50,1.79632,25
6,410,SIVB,730.18,1.64587,2
7,251,IT,332.37,1.62554,6
8,195,FTNT,339.46,1.60867,5
9,23,ALB,244.97,1.58357,8


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [12]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Price Percentice',
    'Six-Month Price Return',
    'Six-Month Price Percentice',
    'Three-Month Price Return',
    'Three-Month Price Percentice',
    'One-Month Price Return',
    'One-Month Price Percentice',
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe =  hqm_dataframe.append(
        pd.Series(
        [
            symbol, 
            data[symbol]['price'],
            'N/A',
            data[symbol]['stats']['year1ChangePercent'],
            'N/A',
            data[symbol]['stats']['month6ChangePercent'],
            'N/A',
            data[symbol]['stats']['month3ChangePercent'],
            'N/A',
            data[symbol]['stats']['month1ChangePercent'],
            'N/A',

        ],
            index= hqm_columns
        ),
        ignore_index = True
        )


In [14]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Price Percentice,Six-Month Price Return,Six-Month Price Percentice,Three-Month Price Return,Three-Month Price Percentice,One-Month Price Return,One-Month Price Percentice
0,A,160.04,N/A,0.48468,N/A,0.172077,N/A,0.0571427,N/A,-0.104648,N/A
1,AAL,19.66,N/A,0.576825,N/A,-0.0963519,N/A,0.0302062,N/A,-0.010524,N/A
2,AAP,226.51,N/A,0.486198,N/A,0.180399,N/A,0.0993747,N/A,0.0901097,N/A
3,AAPL,155.69,N/A,0.292825,N/A,0.110566,N/A,0.0467282,N/A,0.0190623,N/A
4,ABBV,111.88,N/A,0.348129,N/A,0.0125998,N/A,-0.059889,N/A,0.00951855,N/A
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.38,N/A,0.317629,N/A,0.0797017,N/A,0.116142,N/A,-0.0113105,N/A
501,ZBH,152.91,N/A,0.0554925,N/A,-0.137111,N/A,-0.00684696,N/A,0.0170859,N/A
502,ZBRA,532.93,N/A,0.81376,N/A,0.0386395,N/A,0.0438464,N/A,-0.0737252,N/A
503,ZION,66.26,N/A,1.16276,N/A,0.147605,N/A,0.341438,N/A,0.11161,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [16]:
time_periods  = [
    
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month',
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
    
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        if hqm_dataframe.loc[row, change_col] == None:
            hqm_dataframe.loc[row, change_col] = 0.0
            
for row in hqm_dataframe.index:
    for time_period in time_periods:
    
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'

        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])
        


In [17]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Price Percentice,Six-Month Price Return,Six-Month Price Percentice,Three-Month Price Return,Three-Month Price Percentice,One-Month Price Return,One-Month Price Percentice,One-Year Return Percentile,Six-Month Return Percentile,Three-Month Return Percentile,One-Month Return Percentile
0,A,160.04,N/A,0.48468,N/A,0.172077,N/A,0.0571427,N/A,-0.104648,N/A,64.554455,73.861386,50.099010,2.376238
1,AAL,19.66,N/A,0.576825,N/A,-0.0963519,N/A,0.0302062,N/A,-0.010524,N/A,71.881188,13.267327,39.801980,31.881188
2,AAP,226.51,N/A,0.486198,N/A,0.180399,N/A,0.0993747,N/A,0.0901097,N/A,64.752475,76.039604,65.544554,82.376238
3,AAPL,155.69,N/A,0.292825,N/A,0.110566,N/A,0.0467282,N/A,0.0190623,N/A,40.792079,61.188119,46.732673,52.475248
4,ABBV,111.88,N/A,0.348129,N/A,0.0125998,N/A,-0.059889,N/A,0.00951855,N/A,49.702970,38.613861,11.485149,46.138614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.38,N/A,0.317629,N/A,0.0797017,N/A,0.116142,N/A,-0.0113105,N/A,45.544554,55.049505,68.910891,31.485149
501,ZBH,152.91,N/A,0.0554925,N/A,-0.137111,N/A,-0.00684696,N/A,0.0170859,N/A,16.831683,8.712871,24.752475,50.495050
502,ZBRA,532.93,N/A,0.81376,N/A,0.0386395,N/A,0.0438464,N/A,-0.0737252,N/A,86.138614,45.940594,44.950495,6.732673
503,ZION,66.26,N/A,1.16276,N/A,0.147605,N/A,0.341438,N/A,0.11161,N/A,92.871287,68.316832,96.633663,87.326733


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [25]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    

In [27]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Price Percentice,Six-Month Price Return,Six-Month Price Percentice,Three-Month Price Return,Three-Month Price Percentice,One-Month Price Return,One-Month Price Percentice,One-Year Return Percentile,Six-Month Return Percentile,Three-Month Return Percentile,One-Month Return Percentile,HQM Score
0,A,160.04,N/A,0.48468,N/A,0.172077,N/A,0.0571427,N/A,-0.104648,N/A,64.554455,73.861386,50.099010,2.376238,47.722772
1,AAL,19.66,N/A,0.576825,N/A,-0.0963519,N/A,0.0302062,N/A,-0.010524,N/A,71.881188,13.267327,39.801980,31.881188,39.207921
2,AAP,226.51,N/A,0.486198,N/A,0.180399,N/A,0.0993747,N/A,0.0901097,N/A,64.752475,76.039604,65.544554,82.376238,72.178218
3,AAPL,155.69,N/A,0.292825,N/A,0.110566,N/A,0.0467282,N/A,0.0190623,N/A,40.792079,61.188119,46.732673,52.475248,50.297030
4,ABBV,111.88,N/A,0.348129,N/A,0.0125998,N/A,-0.059889,N/A,0.00951855,N/A,49.702970,38.613861,11.485149,46.138614,36.485149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.38,N/A,0.317629,N/A,0.0797017,N/A,0.116142,N/A,-0.0113105,N/A,45.544554,55.049505,68.910891,31.485149,50.247525
501,ZBH,152.91,N/A,0.0554925,N/A,-0.137111,N/A,-0.00684696,N/A,0.0170859,N/A,16.831683,8.712871,24.752475,50.495050,25.198020
502,ZBRA,532.93,N/A,0.81376,N/A,0.0386395,N/A,0.0438464,N/A,-0.0737252,N/A,86.138614,45.940594,44.950495,6.732673,45.940594
503,ZION,66.26,N/A,1.16276,N/A,0.147605,N/A,0.341438,N/A,0.11161,N/A,92.871287,68.316832,96.633663,87.326733,86.287129


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [28]:
hqm_dataframe.sort_values('HQM Score', ascending = False , inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace = True , drop = True)

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [32]:
portfolio_input()

Enter the size of your portfolio:1000000


In [34]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe.loc[i, 'Price'])
    


C:\Users\Aniket Bhanderi\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [35]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Price Percentice,Six-Month Price Return,Six-Month Price Percentice,Three-Month Price Return,Three-Month Price Percentice,One-Month Price Return,One-Month Price Percentice,One-Year Return Percentile,Six-Month Return Percentile,Three-Month Return Percentile,One-Month Return Percentile,HQM Score
0,DVN,41.20,485,3.83132,N/A,0.882191,N/A,0.652287,N/A,0.354904,N/A,100.000000,99.801980,99.801980,99.603960,99.801980
1,MCHP,75.38,265,1.75566,N/A,0.968904,N/A,1.31967,N/A,0.860841,N/A,99.009901,100.000000,100.000000,100.000000,99.752475
2,MRO,16.63,1202,3.17395,N/A,0.5754,N/A,0.503679,N/A,0.367843,N/A,99.603960,99.009901,99.207921,99.801980,99.405941
3,FANG,114.23,175,3.17442,N/A,0.462982,N/A,0.542916,N/A,0.347302,N/A,99.801980,98.217822,99.405941,99.405941,99.207921
4,COP,78.99,253,1.38298,N/A,0.523475,N/A,0.4168,N/A,0.279466,N/A,96.831683,98.811881,98.613861,98.811881,98.267327
5,OXY,34.30,583,2.47138,N/A,0.355032,N/A,0.33822,N/A,0.223434,N/A,99.405941,96.633663,96.435644,96.831683,97.326733
6,PXD,193.98,103,1.26896,N/A,0.315739,N/A,0.435062,N/A,0.230446,N/A,95.247525,93.861386,98.811881,97.623762,96.386139
7,MOS,42.34,472,1.26265,N/A,0.295805,N/A,0.465,N/A,0.254703,N/A,94.851485,91.881188,99.009901,98.217822,95.990099
8,SIVB,703.32,28,1.6092,N/A,0.361855,N/A,0.310789,N/A,0.147884,N/A,98.415842,96.831683,94.653465,93.861386,95.940594
9,OKE,67.71,295,1.57691,N/A,0.308015,N/A,0.330901,N/A,0.185585,N/A,97.821782,93.069307,95.643564,96.435644,95.742574


In [50]:
hqm_dataframe.to_excel("Momentum Strtegy.xlsx",    sheet_name='Sheet-1')  

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [41]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [42]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [48]:
column_formats = {
    'A' :['Ticker', string_template],
    'B' :['Price', dollar_template],
    'C' :['Number of Shares to Buy', integer_template],
    'D' :['One-Year Price Return',percent_template],
    'E' :['One-Year Price Percentice',percent_template],
    'F' :['Six-Month Price Return',percent_template],
    'G' :['Six-Month Price Percentice',percent_template],
    'H' :['Three-Month Price Return',percent_template],
    'I' :['Three-Month Price Percentice',percent_template],
    'J' :['One-Month Price Return',percent_template],
    'K' :['One-Month Price Percentice',percent_template],
    'M' :['Six-Month Return Percentil',percent_template],
    'N' :['Three-Month Return Percentile',percent_template],
    'O' :['One-Month Return Percentile',percent_template],
    'p': ['HQM Score',percent_template],
}


for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}' , 22 , column_formats[column][1])

    writer.save()

KeyError: 'Momentum Strategy'

## Saving Our Excel Output

As before, saving our Excel output is very easy: